第四回レポート課題
参考文献1を参考に誤差逆伝搬法について学習した。\
学習にあたり、まず種々のレイヤの実装を実際に行い、理解を深めることとした

1.計算グラフについての理解\
ニューラルネットワークにおける処理の一つ一つをレイヤという単位で考えることができる。これにより開発が容易になる。
逆伝搬により入力した値による出力値の偏微分の値を効率よく計算できる

2.乗算レイヤの実装

In [1]:
#乗算レイヤ
class MulLayer:
    def __init__(self):
        #入力値の保存用
        self.x = None 
        self.y = None
    
    #順伝播
    def forward(self, x, y):
        #入力値の保存
        self.x = x
        self.y = y
        #乗算
        out = x * y
        return out
    
    #逆伝播
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

In [2]:
#実行
apple = 100
apple_num = 10
tax_rate = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax_rate)

print(price)

#backward
dprice = 1
dapple_price, dtax_rate = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax_rate)

1100.0
11.0 110.00000000000001 1000


確かに計算グラフ通りの値を出力している

3.加算レイヤの実装

In [3]:
class AddLayer():
    def __init__(self):
        pass
    
    def forward(self, x, y):
        #加算
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy    

In [4]:
#実行
apple = 100
apple_num = 10
orange = 150
orange_num = 25
tax_rate = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax_rate)

print(price)

#backward
dprice = 1
dall_price, dtax_rate = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dorange, dorange_num, dtax_rate)


5225.0
11.0 110.00000000000001 27.500000000000004 165.0 4750


このように出力値に対する各変数の偏微分した値をbackward()メソッドで求めることができる
また、forwardとbackwardで処理する順番が真逆であることも確認できた。

活性化関数の実装
非線形な関数についても考える。また、スカラーではなくNumpy配列を扱うことを考える。

4.RELUレイヤの実装\
$$y = x (x >  0)$$
$$y = 0 (x <= 0)$$
なので
$$\frac{\partial y}{\partial x} = 1 (x >  0)$$
$$\frac{\partial y}{\partial x} = 0 (x <= 0)$$
になることに注意\
ReLUレイヤはスイッチのような役割を果たす

In [5]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0) #0以下の場所をTrueとして記録した配列
        out = x.copy() #複製(入力値への影響をなくす(そのまま代入だと参照渡しになるのでcopy()を用いる))
        out[self.mask] = 0 #mask内でTrueの場所に0を代入
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0 #入力値が0以下は0に,0より大きい場合はそのまま
        dx = dout
        return dx

In [6]:
#実行
import numpy as np
x = np.array( [[ 1.0, -0.5], 
               [-2.0,  3.0]])

print(x)

relu_layer = Relu()

#順伝播
y = relu_layer.forward(x)
print(y)

print(relu_layer.mask)

#逆伝播
dout = np.array([[ 1.0, 1.0], 
               [1.0, 1.0]])
dx = relu_layer.backward(dout)
print(dx)

[[ 1.  -0.5]
 [-2.   3. ]]
[[1. 0.]
 [0. 3.]]
[[False  True]
 [ True False]]
[[1. 0.]
 [0. 1.]]


5.Sigmoidレイヤの実装\
Sigmoid関数は次のように表すことができる\
$$y = \frac{1}{1 + exp(-x)}$$
また逆伝播の出力は以下のようになる。導出は参考文献1を参考にした。\
$$\frac{\partial L}{\partial y} y^{2}exp(-x) = \frac{\partial L}{\partial y} y (1 - y)$$

In [7]:
class Sigmoid:
    def __init__(self):
        #出力結果(y)の保存用
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1.0 - self.out)
        return dx

6.Affineレイヤの実装\
重みと入力値の線形和を取り、バイアスを加える処理をnumpy配列で行うことを考える\
逆伝播について、和に対してはスカラーの時同様にそのまま通せばよい。積のときは反対の入力値をかけて通すが行列の場合、行列のサイズに注意して転置して向きを考えてからかける。詳しくは参考文献1参照\
 バイアスの加算については以下の性質(ブロードキャスト)に注意

In [8]:
# 順伝播
xw = np.array([[ 0,  0,  0], 
               [10, 10, 10]])
b = np.array([1, 2, 3])
y = xw + b #bが1×3から2×3にサイズが変わって加算されているこのとき列方向に拡張
print(y)

#逆伝播
dy = np.array([[1, 2, 3], 
               [4, 5, 6]])
db = np.sum(dy)
print(db) 
db = np.sum(dy, axis = 0) #axis = 0 で列方向に処理
print(db)
db = np.sum(dy, axis = 1) #axis = 1 で行方向に処理
print(db)

[[ 1  2  3]
 [11 12 13]]
21
[5 7 9]
[ 6 15]


In [9]:
class Affine:
    def __init__(self, W, b):
        self.W = W #weight
        self.b = b #bias
        self.x = None #入力保存用
        self.dW = None #dW保存
        self.db = None #db保存
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T) #∂L/∂x = ∂L/∂y(=dout)・W^t
        self.dW = np.dot(self.x.T, dout) #∂L/∂w = x^t・∂L/∂y(=dout)
        self.db = np.sum(dout, axis = 0)
        return dx

In [10]:
#実行
x = np.random.rand(5, 2) #5*2
w = np.random.rand(2, 3) #2*3
b = np.random.rand(1, 3) #1*3
#y (2*4)*(4*3) = 5*3
print('x {}'.format(x.shape))
print(x)
print('w {}'.format(w.shape))
print(w)
print('b {}'.format(b.shape))
print(b)

#順伝播
affine_layer = Affine(w, b)
y = affine_layer.forward(x)
print('y {}'.format(y.shape))
print(y) #5*3

#逆伝播
dx = affine_layer.backward(y)
print('dx {}'.format(dx.shape))
print(dx)
print('dW {}'.format(affine_layer.dW.shape))
print(affine_layer.dW)
print('db {}'.format(affine_layer.db.shape))
print(affine_layer.db)

x (5, 2)
[[0.2353646  0.07768556]
 [0.10410011 0.47737009]
 [0.4696064  0.20996369]
 [0.31006524 0.01962232]
 [0.95613636 0.1596323 ]]
w (2, 3)
[[0.1804642  0.82047527 0.60915726]
 [0.92056947 0.95612174 0.08416438]]
b (1, 3)
[[0.6979588  0.7935734  0.28061881]]
y (5, 3)
[[0.81194864 1.06096108 0.43053122]
 [1.15619747 1.33540888 0.3842097 ]
 [0.97599211 1.37962469 0.58435442]
 [0.77197818 1.06673559 0.4711488 ]
 [1.01745981 1.73068755 0.87649156]]
dx (5, 2)
[[1.2792812  1.79809848]
 [1.53836634 2.37351033]
 [1.6640433  2.26673952]
 [1.3015483  1.77024258]
 [2.13752259 2.66515979]]
dW (2, 3)
[[1.9819903  3.02214038 1.39987696]
 [0.99750112 1.30678216 0.48871088]]
db (3,)
[4.7335762  6.57341778 2.74673569]


参考文献1の図5-27に相当する。$N = 5$として考えた場合である。\
サイズに注目すると、慣れないベクトル、行列の偏微分なども分かりやすくなった。\
Affineはニューラルネットワークにおいて活性化関数と交互に出てくる重要なレイヤなのでしっかり理解できてよかった。

7.Softmax-with-Lossレイヤの実装\
正規化して出力するソフトマックス関数と、損失関数交差エントロピー誤差を合わせたレイヤを考える。\
順伝播や逆伝播の導出は煩雑なので結果だけに注目することとした。\
$(a1, a2, a3)$ Softmaxへの入力\
$(y1, y2, y3)$ Softmaxからの出力 & Cross Emtropy Errorへの入力\
$(t1, t2, t3)$ Cross Emtropy Errorへの入力(教師データ)\
とすると、各入力への逆伝播は\
$(y1 - t1, y2 - t2, y3 - t3)$
になる。\
この逆伝播の値はニューラルネットワークと教師データの差であり、精度のようなものであるといえる。したがって逆伝播の値を小さくするように考えていけばよい。

In [11]:
class SoftmaxWithLoss():
    def __init__(self):
        self.loss = None #損失関数保存
        self.y = None #Softmax出力保存
        self.t = None #教師データ保存
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout  = 1):
        batch_size = self.t.shape[0] #入力データ数
        dx = (self.y - self.t) / batch_size #正規化
        return dx

def softmax(a):
    c = np.max(a) #最大値計算
    exp_a = np.exp(a - c) #入力を定数ずらすのは問題ない、最大値分下げることでオーバーフロー対策
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(np.log( y[np.arange(batch_size), t] + 1e-7 ))

In [12]:
#実行
t = np.array([0, 1, 0])
x = np.random.rand(3)
print('t {}'.format(t.shape))
print(t)
print('x {}'.format(x.shape))
print(x)

#順伝播
softmaxwithloss_layer = SoftmaxWithLoss()
l = softmaxwithloss_layer.forward(x, t)
print('loss')
print(l)

#逆伝播
dx = softmaxwithloss_layer.backward()
print('dx {}'.format(dx.shape))
print(dx)

abs_dx = np.abs(dx)
print('abs_dx')
print(abs_dx)
print(np.argmin(abs_dx))

t (3,)
[0 1 0]
x (3,)
[0.22944171 0.72390616 0.37398078]
loss
3.5066879851515314
dx (3,)
[ 0.08783194 -0.18932234  0.1014904 ]
abs_dx
[0.08783194 0.18932234 0.1014904 ]
0


結果を見ると、たしかに逆伝播を用いて精度の評価ができている。\
教師データを[0, 1, 0]として実行すると、損失関数l及び、その偏微分dxが計算できている。\
また、入力の中で教師データと一番近いものをdxの絶対値の大きさを用いて判断出来ている。\
したがってdxをもちいて前回学んだ勾配降下法を行うことができる。

ここまでで各レイヤの理解が深まったので、ここからはサンプルコードを参考に、誤差逆伝播法に対応したニューラルネットワークを実装する。
前回同様、二層で考える。

In [13]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        #weight,biasの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)
        #layer
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()   
        
    def predict(self, x):
        for layer in self.layers.values(): #0層から順にforward処理(lastlayerは行わない)
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t) #損失関数の計算
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1) #行ごとに処理、最大値のインデックスを求める
        if t.ndim != 1:
            t = np.argmax(t, axis = 1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        #forward
        self.loss(x, t) #損失関数の計算
        
        #backward
        dout = 1
        dout = self.lastLayer.backward(dout) #
        
        layers = list(self.layers.values()) #forwardの順にレイヤが格納されたリスト
        layers.reverse() #backwardの順に変更
        for layer in layers:
            dout = layer.backward(dout) #各層のbackward処理を行う,最後まで行くと入力値まで求まる
        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        return grads

In [14]:
from dataset.mnist import load_mnist
#実行
#データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

# 数値微分
grad_numerical = network.numerical_gradient(x_batch, t_batch)
# Backward
grad_backprop = network.gradient(x_batch, t_batch)

#数値微分と誤差逆伝播法により計算された勾配の差の絶対値を計算
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:2.580907633960024e-13
b1:8.865927591150853e-13
W2:9.42440426426102e-13
b2:1.205702274131859e-10


TwoLayerNetクラス内に誤差逆伝播法による勾配計算するメソッドを実装した。\
精度については勾配の差が1e-10以上の精度で求まっており、前回の結果と合わせると二つとも十分であるといえる。\
実行速度について考察する。

In [15]:
import time

numerical_start = time.time()
network.numerical_gradient(x_batch, t_batch)
numerical_time = time.time() - numerical_start

start = time.time()
network.gradient(x_batch, t_batch)
time = time.time() - start

print (time, numerical_time, time - numerical_time)

0.00099945068359375 7.4606852531433105 -7.459685802459717


参考文献6を参考に勾配計算にかかる時間を計測した。timeは誤差逆伝搬法, numerical_timeは数値微分である。\
timeは0.0009となっていて、処理にほとんど時間がかかっていないが、numerical_timeは7.4程度であり、数値微分よりも誤差逆伝搬法のほうが実行速度が速いことが確認できた。今回のデータ数で、
$$\frac{7.4}{0.0009} \simeq 8200$$
となり、およそ8200倍である。データ数が増えればさらにこの差は大きくなると考えられる。

最後に、前回の内容と合わせたまとめとして、誤差逆伝搬法を用いた学習について考える。

In [16]:
import numpy as np
from dataset.mnist import load_mnist

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

#NNの作成
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

#ハイパーパラメータ
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

#各種パラメータ推移記録用リスト
train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 誤差逆伝搬法による勾配計算
    grad = network.gradient(x_batch, t_batch)
    
    # 更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    #accuracy保存
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print('{}, {}, rate = {}'.format(train_acc, test_acc, train_acc - test_acc) )
        
last_acc_dif = abs( train_acc_list[-1] - test_acc_list[-1] )
rate = last_acc_dif / train_acc_list[-1]
print('last_rate = {}'.format(rate))

0.18816666666666668, 0.1865, rate = 0.0016666666666666774
0.90495, 0.9099, rate = -0.00495000000000001
0.9178333333333333, 0.9198, rate = -0.001966666666666672
0.9332333333333334, 0.9335, rate = -0.0002666666666666373
0.9410833333333334, 0.9397, rate = 0.001383333333333403
0.9510166666666666, 0.9482, rate = 0.0028166666666665785
0.95585, 0.953, rate = 0.002850000000000019
0.9592333333333334, 0.9564, rate = 0.0028333333333333544
0.9634333333333334, 0.9587, rate = 0.004733333333333367
0.9674166666666667, 0.9633, rate = 0.004116666666666657
0.9688666666666667, 0.9636, rate = 0.005266666666666642
0.9719, 0.9646, rate = 0.007299999999999973
0.9737166666666667, 0.9667, rate = 0.007016666666666671
0.9736, 0.9664, rate = 0.007199999999999984
0.9764666666666667, 0.9671, rate = 0.009366666666666745
0.9758666666666667, 0.9676, rate = 0.008266666666666644
0.9791, 0.9687, rate = 0.010399999999999965
last_rate = 0.010621999795730738


train, testいずれも最終的な精度は95%を超えていて, 精度の観点では問題はない。また、二つのaccuracyの差は最終的に約0.01であり、約1%認められた。したがってわずかに過学習を起こしているが、問題ない範囲だと考えた。前回の数値微分法と比べても精度の点ではあまり変化はないようである。実行速度の点で優れている。実行速度は大量のデータを扱うこの分野においては非常に重要な要素であるため、多少実装が複雑でも誤差逆伝搬法を用いるべきだといえる。

参考文献
1. ゼロから作る Deep Learning Pythonで学ぶディープラーニングの理論と実装
2. https://qiita.com/mytk0u0/items/231807f4136b2b1681b0 (copyメソッドについて調べるのに利用)
3. https://tutorials.chainer.org/ja/05_Basics_of_Linear_Algebra.html (行列の偏微分について調べるのに利用)
4. https://note.nkmk.me/python-numpy-transpose/ (numpy配列の転置について調べるのに利用)
5. https://www.sejuku.net/blog/70994 (numpyでの絶対値の使い方を調べるのに利用)
6. https://qiita.com/fantm21/items/3dc7fbf4e935311488bc (Python処理にかかる時間について調べるのに利用)